In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    Create a dataframe to be used as the base of a power prediction model
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')
train_start_time = pd.to_datetime('2014-03-31')
train_end_time = pd.to_datetime('2014-10-01')
test_end_time = pd.to_datetime('2014-11-01')


#infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_jobs_trainset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_trainset"
infile_jobs_testset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_testset"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [2]:
jobs_train = pd.read_csv(infile_jobs_trainset + ".csv", index_col=0)
print("train set contains {} records".format(jobs_train.shape[0]))
jobs_test = pd.read_csv(infile_jobs_testset + ".csv", index_col=0)
print("test set contains {} records".format(jobs_test.shape[0]))

train set contains 146901 records
test set contains 18224 records


In [3]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))
### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records
after the clean, jobs_to_nodes_whole_data contains 444610 records


In [4]:
###### NB eseguire solo se non si vogliono job sotto 5 sec

#jobs_train = jobs_train[pd.to_datetime(jobs_train['end_time']) - pd.to_datetime(jobs_train['run_start_time']) > np.timedelta64(5, 's')]
#print(jobs_train.shape[0])
#jobs_test = jobs_test[pd.to_datetime(jobs_test['end_time']) - pd.to_datetime(jobs_test['run_start_time']) > np.timedelta64(5, 's')]
#print(jobs_test.shape[0])

In [5]:
# train_whole: 167324 ; test_whole:29935 
# train_long: 39363 ; test_long: 11622

In [6]:
### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(jobs_train, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

merged_jobs_to_nodes contains 161420 records


In [7]:
jobs_to_nodes_whole_data.head(1)

,node_id,job_id_string,ncpus,ngpus,nmics,mem_requested
job2node_id,,,,,,
1,17,498458.node129,16,0,0,14680064


In [8]:
jobs_train.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes,ncores_system,njobs_system,load_system,pred_perNode,pred_perUser
job_id,,,,,,,,,,,,,,,,,,,,,
2,498459.node129,CPMD,parallel,2014-03-31 00:36:51,2014-03-31 00:36:51,2014-03-31 00:40:13,framondo,1,16,14,...,0,0,40,1,1,32,2,0,242.50631,375.626147


### good code

We want to obtain 5 dataframes:
- node
- user
- user_node
- user_job
- user_job_node

In [20]:
merged_jobs_to_nodes['core_avg'] = merged_jobs_to_nodes['real_pow'] / merged_jobs_to_nodes['cpu_req']

In [21]:
# user_jobName_node
df_user_job_node = merged_jobs_to_nodes.groupby(['user', 'job_name', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user_job_node.shape[0])

# Drop a row by condition
df_user_job_node = df_user_job_node[df_user_job_node['mean_core'] != 0.0]
df_user_job_node.head(100)

22730


,user,job_name,node_id,mean_core,count
0,a07smr01,A2REPO,7,12.412156,1
1,a07smr01,A2REPO,33,3.937672,1
2,a07smr01,A2REPO,35,4.870611,53
3,a07smr01,A2REPO,46,2.749792,21
4,a07smr01,ARCHIVE,7,11.895912,1
5,a07smr01,ARKI,33,5.766090,7
6,a07smr01,ARKI,35,4.523225,77
7,a07smr01,ARKI,46,4.146008,26
8,a07smr01,INIT,33,4.631090,4
9,a07smr01,INIT,35,4.043560,68


In [22]:
outfile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
#outfile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_long"
df_user_job_node.to_csv(outfile_user_job_node + ".csv")

In [11]:
jobs_train.shape

(146901, 31)

In [10]:
# timing
import time

start_time = time.time()
merged_jobs_to_nodes = pd.merge(jobs_train, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("--- %s seconds ---" % (time.time() - start_time))
data_preparation_time = time.time() - start_time

start_time = time.time()
merged_jobs_to_nodes['core_avg'] = merged_jobs_to_nodes['real_pow'] / merged_jobs_to_nodes['cpu_req']
# user_jobName_node
df_user_job_node = merged_jobs_to_nodes.groupby(['user', 'job_name', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
#print(df_user_job_node.shape[0])

# Drop a row by condition
df_user_job_node = df_user_job_node[df_user_job_node['mean_core'] != 0.0]
print("--- %s seconds ---" % (time.time() - start_time))
training_time = time.time() - start_time

print("total: {}".format(data_preparation_time + training_time))

--- 0.3950357437133789 seconds ---
--- 0.10303926467895508 seconds ---
total: 0.498349666595459


In [23]:
# user_jobName
df_user_job = merged_jobs_to_nodes.groupby(['user', 'job_name']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user_job.shape[0])
# Drop a row by condition
df_user_job = df_user_job[df_user_job['mean_core'] != 0.0]
df_user_job.head(5)

6491


,user,job_name,mean_core,count
0,a07smr01,A2REPO,4.371550,76
1,a07smr01,ARCHIVE,11.895912,1
2,a07smr01,ARKI,4.513156,110
3,a07smr01,INIT,4.865106,95
4,a07smr01,LM28_OPE,22.266872,1164


In [24]:
outfile_user_job = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_whole"
#outfile_user_job = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_whole_long"
df_user_job.to_csv(outfile_user_job + ".csv")

In [25]:
# user_node
df_user_node = merged_jobs_to_nodes.groupby(['user', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user_node.shape[0])
# Drop a row by condition
df_user_node = df_user_node[df_user_node['mean_core'] != 0.0]
df_user_node.head(5)

4783


,user,node_id,mean_core,count
0,a07smr01,1,15.386652,5
1,a07smr01,2,18.405537,4
2,a07smr01,6,18.405537,4
3,a07smr01,7,28.234849,3
4,a07smr01,33,7.076942,40


In [26]:
outfile_user_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_node_whole"
#outfile_user_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_node_whole_long"
df_user_node.to_csv(outfile_user_node + ".csv")

In [27]:
# user
df_user = merged_jobs_to_nodes.groupby(['user']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user.shape[0])
# Drop a row by condition
df_user = df_user[df_user['mean_core'] != 0.0]
df_user.head(5)

263


,user,mean_core,count
0,a07smr01,16.320424,5694
1,a08tra01,28.721584,12
2,a08tra11,108.508580,3
3,a08tra35,14.327342,8
4,a08tra37,13.757758,1


In [28]:
outfile_user = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_whole"
#outfile_user = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_whole_long"
df_user.to_csv(outfile_user + ".csv")

In [29]:
# node
df_node = merged_jobs_to_nodes.groupby(['node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_node.shape[0])
# Drop a row by condition
df_node = df_node[df_node['mean_core'] != 0.0]
df_node.head(5)

63


,node_id,mean_core,count
0,1,9.574410,667
1,2,8.739935,109
2,3,17.070914,806
3,4,16.378783,941
4,5,18.866071,368


In [30]:
outfile_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_node_whole"
#outfile_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_node_whole_long"
df_node.to_csv(outfile_node + ".csv")